<a href="https://colab.research.google.com/github/MilindMali0339/Epileptic-Seizure-Detection-using-ML/blob/main/FCM%2BComplecity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install scikit-fuzzy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 994.0/994.0 kB 7.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-fuzzy: filename=scikit_fuzzy-0.4.2-py3-none-any.whl size=894078 sha256=4acc9f5d1a25937131b9893e516906abf10a149cfdf993cb9f83fa1463627ac1
  Stored in directory: /root/.cache/pip/wheels/4f/86/1b/dfd97134a2c8313e519bcebd95d3fedc7be7944db022094bc8
Successfully built scikit-fuzzy


In [ ]:
pip install --upgrade scikit-fuzzy

In [ ]:
import time
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from scipy.signal import butter, filtfilt
from scipy.stats import kurtosis, skew
from scipy.signal import welch
import skfuzzy as fuzz

# Load EEG signal dataset
filenam = '/content/s00.csv';
def load_data(filenam):
    start_time = time.time()
    eeg_data = pd.read_csv('/content/s00.csv')  # Load data from CSV file
    end_time = time.time()
    print(f"Data loading time: {end_time - start_time} seconds")
    return eeg_data

# Define bandpass filter function
def bandpass_filter(data, lowcut, highcut, fs, order=5):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band')
    return filtfilt(b, a, data)

# Apply bandpass filter to each channel and create filtered_signal columns
def preprocess_data(data):
    start_time = time.time()
    filtered_signals = {}
    for i, col in enumerate(data.columns[1:], 1):  # Start from index 1 to skip the first column
        print(f"Filtering data for {col}...")
        filtered_signals[f'filtered_signal_{i}'] = bandpass_filter(data[col], lowcut=0.5, highcut=50, fs=1000)
    end_time = time.time()
    print(f"Data preprocessing time: {end_time - start_time} seconds")
    return pd.DataFrame(filtered_signals)

# Function to extract time domain features from a signal
def extract_time_domain_features(signal):
    start_time = time.time()
    features = {}
    features['mean'] = np.mean(signal)
    features['std'] = np.std(signal)
    features['max'] = np.max(signal)
    features['min'] = np.min(signal)
    features['median'] = np.median(signal)
    features['kurtosis'] = kurtosis(signal)
    features['skewness'] = skew(signal)
    end_time = time.time()
    print(f"Time domain feature extraction time: {end_time - start_time} seconds")
    return features

# Function to extract frequency domain features from a signal
def extract_frequency_domain_features(signal, fs):
    start_time = time.time()
    f, Pxx = welch(signal, fs=fs, nperseg=fs*2, noverlap=fs)
    features = {}
    features['spectral_centroid'] = np.sum(f * Pxx) / np.sum(Pxx)
    features['spectral_entropy'] = -np.sum(Pxx * np.log2(Pxx))
    features['total_power'] = np.sum(Pxx)
    end_time = time.time()
    print(f"Frequency domain feature extraction time: {end_time - start_time} seconds")
    return features

# Extract features from each filtered channel
def extract_features(filtered_signals):
    start_time = time.time()
    extracted_features = {}
    for i, (col, signal) in enumerate(filtered_signals.items(), 1):
        print(f"Extracting features for {col}...")
        time_features = extract_time_domain_features(signal)
        frequency_features = extract_frequency_domain_features(signal, fs=1000)
        features = {**time_features, **frequency_features}
        extracted_features[f'channel_{i}'] = features
    end_time = time.time()
    print(f"Feature extraction time: {end_time - start_time} seconds")
    return pd.DataFrame(extracted_features).T

# Perform Fuzzy C-Means clustering on the extracted features
def perform_clustering(X, num_clusters):
    start_time = time.time()
    cntr, _, _, _, _, _, _ = fuzz.cluster.cmeans(
        X.T, num_clusters, 2, error=0.005, maxiter=1000)
    end_time = time.time()
    print(f"Clustering time: {end_time - start_time} seconds")
    return cntr

# Assign data points to clusters based on fuzzy clustering centers
num_clusters = 2;
def assign_to_clusters(X, cntr):
    start_time = time.time()
    cntr, u, u0, d, jm, p, fpc = fuzz.cluster.cmeans(
    X.T, num_clusters, 2, error=0.005, maxiter=1000)
    end_time = time.time()
    print(f"Assigning to clusters time: {end_time - start_time} seconds")
    return np.argmax(u, axis=0)

# Train KNN model
def train_knn(X_train, y_train, n_neighbors=5):
    start_time = time.time()
    knn_model = KNeighborsClassifier(n_neighbors=n_neighbors)
    knn_model.fit(X_train, y_train)
    end_time = time.time()
    print(f"KNN training time: {end_time - start_time} seconds")
    return knn_model

# Main function
def main():
    start_time = time.time()
    # Load data
    eeg_data = load_data('s00.csv')  # Replace 's00.csv' with your filename

    # Preprocess data (filtering)
    filtered_signals = preprocess_data(eeg_data)

    # Extract features
    X = extract_features(filtered_signals)

    # Perform clustering
    num_clusters = 2  # You can adjust the number of clusters
    cntr = perform_clustering(X, num_clusters)

    # Assign data points to clusters
    y_clusters = assign_to_clusters(X, cntr)

    # Train KNN model on cluster assignments
    X_train, X_test, y_train, y_test = train_test_split(X, y_clusters, test_size=0.2, random_state=42)
    knn_model = train_knn(X_train, y_train)

    # Evaluate model
    print("Test Set Performance:")
    y_pred = knn_model.predict(X_test)
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print("Classification Report:\n", classification_report(y_test, y_pred))

    # Confusion Matrix
    print("Confusion Matrix:\n", pd.crosstab(y_test, y_pred, rownames=['Actual'], colnames=['Predicted'], margins=True))

    # Predict cluster assignments for test set features
    y_pred_clusters = knn_model.predict(X_test)

    # Define decision rule (example: majority vote)
    # Assuming cluster 0 represents non-seizure and cluster 1 represents seizure
    seizure_detected = (y_pred_clusters == 1).any()

    # Print result
    if seizure_detected:
        print("Seizure detected.")
    else:
        print("No seizure detected.")

    end_time = time.time()
    print(f"Total execution time(Computation complexity): {end_time - start_time} seconds")


if __name__ == "__main__":
    main()



Data loading time: 0.15334558486938477 seconds
Filtering data for -2.7403...
Filtering data for -2.5017...
Filtering data for 0.095121...
Filtering data for -7.0917...
Filtering data for -0.42704...
Filtering data for -2.5918...
Filtering data for -3.3238...
Filtering data for 0.9995...
Filtering data for 0.87011...
Filtering data for -0.080058...
Filtering data for 2.8075...
Filtering data for -0.52398...
Filtering data for 1.9341...
Filtering data for -3.7614...
Filtering data for -1.144...
Filtering data for 0.033687...
Filtering data for 2.7164...
Filtering data for 0.84831...
Data preprocessing time: 0.10546684265136719 seconds
Extracting features for filtered_signal_1...
Time domain feature extraction time: 0.0057964324951171875 seconds
Frequency domain feature extraction time: 0.0023353099822998047 seconds
Extracting features for filtered_signal_2...
Time domain feature extraction time: 0.004986286163330078 seconds
Frequency domain feature extraction time: 0.002103090286254883 s